In [ ]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import scrapbook as sb
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from reco_utils.recommender.rbm.rbm import RBM
from reco_utils.dataset.python_splitters import numpy_stratified_split
from reco_utils.dataset.sparse import AffinityMatrix


from reco_utils.dataset import movielens
from reco_utils.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

#For interactive mode only
%load_ext autoreload
%autoreload 2
%matplotlib inline

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))


load data

In [ ]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

In [ ]:
#data = movielens.load_pandas_df(
#    size=MOVIELENS_DATA_SIZE,
#    header=['userID','movieID','rating','timestamp']
#)
Retrievalv3= 'https://raw.githubusercontent.com/b21-cap0428/machine-learning/main/RetrievalV4.csv'
data = pd.read_csv(Retrievalv3)

# Convert to 32-bit in order to reduce memory consumption 
data.loc[:, 'rating'] = data['rating'].astype(np.int32) 

data.head()

data splitting

In [ ]:
#to use standard names across the analysis 
header = {
        "col_user": "userID",
        "col_item": "movieID",
        "col_rating": "rating",
    }

#instantiate the sparse matrix generation  
am = AffinityMatrix(DF = data, **header)

#obtain the sparse matrix 
X, _, _ = am.gen_affinity_matrix()

In [ ]:
Xtr, Xtst = numpy_stratified_split(X)

In [ ]:
print('train matrix size', Xtr.shape)
print('test matrix size', Xtst.shape)

Training

In [ ]:
#First we initialize the model class
model = RBM(hidden_units= 1000, training_epoch = 150, minibatch_size= 60, keep_prob=0.9,with_metrics =True)

In [ ]:
#Model Fit
train_time= model.fit(Xtr, Xtst)

predicting

In [ ]:
#number of top score elements to be recommended  
K = 10

#Model prediction on the test set Xtst. 
top_k, test_time =  model.recommend_k_items(Xtst)

In [ ]:
top_k_df = am.map_back_sparse(top_k, kind = 'prediction')
test_df = am.map_back_sparse(Xtst, kind = 'ratings')

In [ ]:
top_k_df = am.map_back_sparse(top_k, kind = 'prediction')
test_df = am.map_back_sparse(Xtst, kind = 'ratings')

In [ ]:
top_k_df.sort_values('prediction', ascending = False).loc[top_k_df['userID'] == 0]

Evaluation Metrics

In [ ]:
def ranking_metrics(
    data_size,
    data_true,
    data_pred,
    time_train,
    time_test,
    K
):

    eval_map = map_at_k(data_true, data_pred, col_user="userID", col_item="movieID", 
                    col_rating="rating", col_prediction="prediction", 
                    relevancy_method="top_k", k= K)

    eval_ndcg = ndcg_at_k(data_true, data_pred, col_user="userID", col_item="movieID", 
                      col_rating="rating", col_prediction="prediction", 
                      relevancy_method="top_k", k= K)

    eval_precision = precision_at_k(data_true, data_pred, col_user="userID", col_item="movieID", 
                               col_rating="rating", col_prediction="prediction", 
                               relevancy_method="top_k", k= K)

    eval_recall = recall_at_k(data_true, data_pred, col_user="userID", col_item="movieID", 
                          col_rating="rating", col_prediction="prediction", 
                          relevancy_method="top_k", k= K)

    
    df_result = pd.DataFrame(
        {   "Dataset": data_size,
            "K": K,
            "MAP": eval_map,
            "nDCG@k": eval_ndcg,
            "Precision@k": eval_precision,
            "Recall@k": eval_recall,
            "Train time (s)": time_train,
            "Test time (s)": time_test
        }, 
        index=[0]
    )
    
    return df_result

In [ ]:
eval_100k= ranking_metrics(
    data_size = "mv 100k",
    data_true =test_df,
    data_pred =top_k_df,
    time_train=train_time,
    time_test =test_time,
    K =10)

eval_100k

In [ ]:
# Record results with papermill for tests
#sb.glue("map", eval_100k['MAP'][0])
#sb.glue("ndcg", eval_100k['nDCG@k'][0])
#sb.glue("precision", eval_100k['Precision@k'][0])
#sb.glue("recall", eval_100k['Recall@k'][0])
#sb.glue("train_time", train_time)
#sb.glue("test_time", test_time)

In [ ]:
import firebase_admin as fb
from firebase_admin import firestore
import json

In [ ]:
cred = fb.credentials.Certificate('credential.json')
fb.initialize_app(cred, {'databaseURL': 'https://keenam-cap0428-default-rtdb.asia-southeast1.firebasedatabase.app'})

In [ ]:
db = firestore.client()

In [ ]:
user_db

In [ ]:
user_db.set({
    'cobain': 'bisa'
})

In [ ]:
unique_user=top_k_df['userID'].unique()
for i in unique_user:
    doc_name='User '+str(i)
    user_db = db.collection(u'test_user').document(doc_name)
    df_to_convert=top_k_df.loc[top_k_df['userID'] == i].sort_values('prediction', ascending = False)
    json_dict={}
    for j in range(len(df_to_convert)):
        json_dict[df_to_convert['movieID'].values[j]]=df_to_convert['prediction'].values[j]
    user_db.set(json_dict)